In [16]:
# --- Download ETH/USD daily data ---
import yfinance as yf
import pandas as pd
import numpy as np
import os, requests

In [7]:
from ecb import get_proxies 

In [8]:
# Option 1: full history
# eth = yf.download("ETH-USD", period="max", interval="1d", auto_adjust=False, progress=False)

# (Alternatively specify dates)
eth = yf.download(
    "ETH-USD", 
    start="2020-01-01", 
    end="2025-10-01", 
    interval="1d", 
    proxy=get_proxies(),   # use custom session
    )

C:\Users\lopardo\AppData\Local\Temp\ipykernel_15912\4142478803.py:5: DeprecationWarning: Set proxy via new config function: yf.set_config(proxy=proxy)
  eth = yf.download(
C:\Users\lopardo\AppData\Local\Temp\ipykernel_15912\4142478803.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  eth = yf.download(
[*********************100%***********************]  1 of 1 completed


In [9]:
print(eth.head())
print(eth.tail())
print("Rows:", len(eth))

# Save if needed
eth.to_csv("data/eth_usd_daily_yf.csv")

Price            Close        High         Low        Open       Volume
Ticker         ETH-USD     ETH-USD     ETH-USD     ETH-USD      ETH-USD
Date                                                                   
2020-01-01  130.802002  132.835358  129.198288  129.630661   7935230330
2020-01-02  127.410179  130.820038  126.954910  130.820038   8032709256
2020-01-03  134.171707  134.554016  126.490021  127.411263  10476845358
2020-01-04  135.069366  136.052719  133.040558  134.168518   7430904515
2020-01-05  136.276779  139.410202  135.045624  135.072098   7526675353
Price             Close         High          Low         Open       Volume
Ticker          ETH-USD      ETH-USD      ETH-USD      ETH-USD      ETH-USD
Date                                                                       
2025-09-26  4035.887939  4069.168701  3868.433594  3868.690918  47873508328
2025-09-27  4018.658203  4038.451660  3975.758057  4035.914551  20382555776
2025-09-28  4141.476562  4143.003906  3969.7

In [ ]:
# Flatten MultiIndex columns: keep only first level (Close, High, Low, Open, Volume)
if isinstance(eth.columns, pd.MultiIndex):
    # drop the constant ticker level
    if 'Ticker' in eth.columns.names:
        eth = eth.droplevel('Ticker', axis=1)
    else:
        eth = eth.droplevel(-1, axis=1)
# Lowercase column names
eth.columns = [c.lower() for c in eth.columns]

# Put date into a column
eth = eth.reset_index().rename(columns={'Date': 'date'})

# Reorder/select desired columns
cols = ['date', 'close', 'high', 'low', 'open', 'volume']
eth = eth[cols]

print(eth.head())
print(eth.tail())
print("Rows:", len(eth))
# Save
eth.to_csv("data/eth_usd_daily.csv", index=False)
# eth.to_excel("data/eth_usd_daily.xlsx", sheet_name="Price", index=False)

        date       close        high         low        open       volume
0 2020-01-01  130.802002  132.835358  129.198288  129.630661   7935230330
1 2020-01-02  127.410179  130.820038  126.954910  130.820038   8032709256
2 2020-01-03  134.171707  134.554016  126.490021  127.411263  10476845358
3 2020-01-04  135.069366  136.052719  133.040558  134.168518   7430904515
4 2020-01-05  136.276779  139.410202  135.045624  135.072098   7526675353
           date        close         high          low         open  \
2095 2025-09-26  4035.887939  4069.168701  3868.433594  3868.690918   
2096 2025-09-27  4018.658203  4038.451660  3975.758057  4035.914551   
2097 2025-09-28  4141.476562  4143.003906  3969.792969  4018.659668   
2098 2025-09-29  4217.341797  4234.782715  4087.927246  4141.356445   
2099 2025-09-30  4145.957520  4238.671387  4095.443604  4217.055176   

           volume  
2095  47873508328  
2096  20382555776  
2097  24631307054  
2098  38560429932  
2099  37679153330  
Rows: 210

In [17]:
# Build volatility (Range% and Parkinson)
vol = eth[['date', 'high', 'low', 'close']].copy()
vol['Range'] = (vol['high'] - vol['low']) / vol['close'] * 100
vol['Parkinson'] = np.sqrt((np.log(vol['high'] / vol['low']) ** 2) / (4 * np.log(2)))
vol = vol[['date', 'Range', 'Parkinson']]

os.makedirs("data", exist_ok=True)
with pd.ExcelWriter("data/eth_usd_daily.xlsx", engine="openpyxl") as writer:
    eth.to_excel(writer, sheet_name="Price", index=False)
    vol.to_excel(writer, sheet_name="Volatility", index=False)

In [22]:
intraday = yf.download("ETH-USD", period="60d", interval="15m", progress=False, group_by=None)
intraday.head()

C:\Users\lopardo\AppData\Local\Temp\ipykernel_15912\1444231843.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  intraday = yf.download("ETH-USD", period="60d", interval="15m", progress=False, group_by=None)


Ticker                         ETH-USD                                         \
Price                             Open         High          Low        Close   
Datetime                                                                        
2025-08-03 00:00:00+00:00  3391.687988  3408.576660  3391.687988  3402.028076   
2025-08-03 00:15:00+00:00  3400.241699  3407.477295  3373.355957  3377.059082   
2025-08-03 00:30:00+00:00  3377.193604  3395.547852  3358.148926  3395.547852   
2025-08-03 00:45:00+00:00  3392.255859  3408.414062  3392.255859  3407.302734   
2025-08-03 01:00:00+00:00  3408.376953  3414.197754  3403.459961  3413.396973   

Ticker                                 
Price                          Volume  
Datetime                               
2025-08-03 00:00:00+00:00   281460736  
2025-08-03 00:15:00+00:00   175435776  
2025-08-03 00:30:00+00:00  1339566080  
2025-08-03 00:45:00+00:00   516001792  
2025-08-03 01:00:00+00:00   569868288